In [1]:
import pandas as pd
import numpy as np
import os
import time
from rdkit import Chem
from fp_gen import KlekFPGenerator, MACCSFPGenerator, SubFPGenerator

In [10]:
generator = KlekFPGenerator(n_jobs=16)
data = pd.read_csv('transformer_data/smiles.csv', header=None, chunksize=5000, names=['smiles'])

In [11]:
def could_be_valid(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

In [12]:
def sparse_to_dense(sparse):
    return np.nonzero(sparse)[0].tolist()

In [15]:
for i, chunk in enumerate(data):
    print(i)
    chunk = chunk[chunk['smiles'].apply(could_be_valid)]
    mols = list(map(Chem.MolFromSmiles, chunk['smiles']))
    fps = generator.transform(mols)
    chunk['fps'] = list(map(sparse_to_dense, fps))
    if i == 0:
        chunk.to_csv('transformer_data/chembl_klek.csv', index=False)
    with open('transformer_data/chembl_klek.csv', 'a') as f:
        chunk.to_csv(f, header=False, index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


[18:24:47] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 10 11 15 16 17 19 20 21
/tmp/ipykernel_24899/4090809836.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['fps'] = list(map(sparse_to_dense, fps))


42


In [22]:
reload_data = pd.read_csv('transformer_data/chembl_klek.csv')
reload_data['fps'] = reload_data['fps'].apply(eval)
reload_data.head()

,smiles,fps
0,Cc1ccc(C(=O)Nc2ccc(S(=O)(=O)N=C(N)N)cc2)cc1,"[668, 676, 838, 1012, 1023, 1145, 1192, 1317, ..."
1,COc1ccc(/C=C/C(C)=O)c(OCC=C(C)C)c1,"[125, 132, 296, 646, 668, 1192, 1262, 1644, 25..."
2,O=C(O)/C=C\CCCCCn1ccnc1,"[125, 132, 135, 296, 337, 340, 343, 345, 346, ..."
3,CC(C(=O)NO)N(Cc1ccccc1[N+](=O)[O-])C(=O)Nc1ccc...,"[0, 13, 19, 97, 101, 296, 609, 668, 671, 676, ..."
4,O=C(Cc1ccncc1)NCCNC1c2ccc(Cl)cc2CCc2cccnc21,"[0, 296, 340, 382, 386, 389, 466, 479, 482, 50..."
